## Dependencies

In [ ]:
!pip install clean-text

In [ ]:
import numpy as np
import pandas as pd
import re
from cleantext import clean
import json
import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')


## Parsing JSON File

In [38]:
#! load data given file path and number of line to load
def load_data(file_path,num_lines):
    data = []
    with open(file_path, 'r') as file: 
        for _ in range(num_lines):
            data.append(json.loads(file.readline())) 
    return data

In [ ]:
data=load_data('PIZZA_train.json',2000000)
print(data[:5])

In [ ]:
#! split json data
def get_training_data(data):
    training_data = []
    training_exr=[]
    training_top=[]
    training_top_dec=[]
    for item in data:
        training_data.append(item['train.SRC'])
        training_exr.append(item['train.EXR'])
        training_top.append(item['train.TOP'])
        training_top_dec.append(item['train.TOP-DECOUPLED'])
    return training_data,training_exr,training_top,training_top_dec
training_data,training_exr,training_top,training_top_dec=get_training_data(data)
print(training_data[:2])
print(training_exr[:2])
print(training_top[:2])
print(training_top_dec[:2])

In [41]:
#! a function to handle negations
def handle_negations(text):
    #! negations are based on training set
    negations_pattern = r"\b(?:no|not|without)\s+.*?\b(?=(?:[^\w\s]|$))"
    # print(re.findall(negations_pattern, text))
    text = re.sub(negations_pattern, lambda x: ' '.join([f'not_{word}' for word in x.group(0).split()]), text)
    return text

In [42]:
#!# lemmatize words with all possible pos tags
def lem_word(word):
    possible_pos = [wordnet.NOUN, wordnet.VERB, wordnet.ADJ, wordnet.ADV]
    lemmatizer = WordNetLemmatizer()
    for pos in possible_pos:
        word=lemmatizer.lemmatize(word,pos)
    return word


In [43]:
lemmatizer = WordNetLemmatizer()
#! stopwords list (adding not_)
stopwords = set(stopwords.words('english'))
stopwords.add('like')
not_stopwords = ['not_' + word for word in stopwords]
stopwords.update(not_stopwords)
# stopwords.discard('a')
# stopwords.discard('an')
# stopwords.discard('not')
# stopwords.discard('no')
# stopwords.discard('can')
# stopwords.discard('not_a')
# stopwords.discard('not_an')
# stopwords.discard('not_can')
# stopwords.discard('not_no')
stopwords=set()
# stopwords.add('and')
# stopwords.add('also')

In [ ]:
#! tokenize training data and remove stop words
def preprocess_training_data(training_data, stopwords):
    # training_data = [handle_negations(order) for order in training_data]
    training_data = [word_tokenize(order) for order in training_data]
    training_data = [[word.lower() for word in order if word.lower() not in stopwords] for order in training_data]
    training_data = [clean(order, no_line_breaks=True, no_punct=True, no_currency_symbols=True) for order in training_data]
    print(training_data)
    #! remove d letter most probably garbage
    training_data = [re.sub(r'\bd\s+', '', order) for order in training_data]
    #! remove "can" at the beginning of the sentence
    # training_data=[re.sub(r'^can\s+', '', order) for order in training_data]
    training_data = [word_tokenize(order) for order in training_data]
    # training_data = [[lemmatizer.lemmatize(word) for word in order] for order in training_data]
    training_data = [[lem_word(word) for word in order] for order in training_data]
    return training_data
training_data = preprocess_training_data(training_data, stopwords)
print(training_data[:5])

In [45]:
#! save processed training dataset
with open('training_data_processed.txt', 'w') as f:
    for item in training_data:
        f.write("%s\n" % item)

In [46]:
#! a utility function for extra parentheses ) removal 
#! handles COMPLEX_TOPPING, NOT,... parenthesis cases
def remove_unmatched_parentheses(input_string):
    result = list(input_string)  # Convert to list for mutability
    last_bracket_index=-1
    for i, char in enumerate(result):
        if char == ')' and i+2 < len(result):
            result[i] = ''  
            last_bracket_index=i
        elif char == '(':
            if last_bracket_index!=-1:
                result[last_bracket_index] = ')'
                last_bracket_index=-1
        elif char == ')' and i+2 >= len(result):
            result[i] = ''
            result[last_bracket_index] = ')'
    return ''.join(result)

In [ ]:
#! get PIZZAORDER, DRINKORDER, NONE Labels 0=>PIZZAORDER, 1=>DRINKORDER, 2=>NONE

def get_order_category_labels(training_top, training_data, stopwords):
    order_category_labels = []
    for i, item in enumerate(training_top):
        order_category_labels.append([2] * len(training_data[i]))
        unwanted_keywords = r"\b(ORDER|SIZE|STYLE|TOPPING|COMPLEX_TOPPING|QUANTITY|NOT|NUMBER|DRINKTYPE|CONTAINERTYPE|VOLUME)\b"
        cleaned_string = re.sub('\('+unwanted_keywords, "", item)
        cleaned_string = [word for word in cleaned_string.split() if word.lower() not in stopwords]
        cleaned_string = ' '.join(cleaned_string)
        cleaned_string = remove_unmatched_parentheses(cleaned_string)
        order_regex = r"\((?:PIZZAORDER|DRINKORDER).*?\)"
        extracted_orders = re.findall(order_regex, cleaned_string)
        k = 0
        for order in extracted_orders:
            order = re.sub(r"[\(\)]", "", order)
            order=word_tokenize(order) #! fix id and don't bugs
            order=clean(order, no_line_breaks=True, no_punct=True, no_currency_symbols=True)
            order=re.sub(r'\bd\s+', '', order) #! for d removal
            tokens = word_tokenize(order)
            # tokens = [lemmatizer.lemmatize(word) for word in tokens]
            tokens = [lem_word(word) for word in tokens]
            j = 0
            to_index_train=training_data[i][k:]
            if 'pizzaorder' in tokens:
                tokens.remove('pizzaorder')
                for word in to_index_train:
                    if j == len(tokens):
                        break
                    if word == tokens[j]:
                        order_category_labels[i][k] = 0
                        j += 1
                    k += 1
            elif 'drinkorder' in tokens:
                tokens.remove('drinkorder')
                for word in to_index_train:
                    if j == len(tokens):
                        break
                    if word == tokens[j]:
                        order_category_labels[i][k] = 1
                        j += 1
                    k += 1
    return order_category_labels

order_category_labels = get_order_category_labels(training_top, training_data, stopwords)
print(order_category_labels[:5])
    

In [48]:
for labels in order_category_labels:
    with open('order_category_labels.txt', 'a') as f:
        f.write("%s\n" % labels)